## Why am I writing this?

Well I was revising concepts in Machine Learning. And if you have ever done so yourself, you might agree it won't go without taking a dive into PCA, why  is it, what is it and how is it used. 

### Let's first talk about Why PCA?

Consider a rock band that is made up of 20 members. That's too many members for a band isn't it. Maybe yes, maybe no. Let's say the band has 3 guitarists, couple of vocalists, one on piano, one on an electronic keyboard, some have flute etc. The band size might work okay depending on the venue like Madison-square or Wembley. But what if the only available venues for this band are coffee shops. Well in that case, 20 members just might seem too many. What seems appropriate for the coffee shops is having like 1 vocalist, 1 on drums, 1 on guitar and may be 1 can play keyboard. You might not get all the intricate details of a song but it just might be good enough for the Coffee shop to listen to in an almos MTV Unplugged kind od music. 

This is exactly the purpose of PCA (Principal Component Analysis) where the band members are the variables representing the data and the song is what the data is representing.

#### With PCA, we are trying to find an alternate representation of data with lesser number of variables such that we are able to capture maximum variance in the data.


### What is PCA?

PCA reduces the dimensionality of input by combining the input variable in such a way that it maximizes variance. Essentially, it is looking to project inputs along an axis maximizing variance. Okay, I've mentioned this multiple times now that we are looking to maximize variance but what does that mean. In simple terms, we are looking to capture most information with lesser variable. Before diving deeper, it is important to have the inputs scaled, and by scaling I mean to subract mean from the input and dividing by standard deviation. As we follow on the math, it would start making sense why that is important. 

So let get into this now.
Consider X be the input matrix of n samples and m variables (nxm). 
We are looking to find a representation of the inputs t such that 

max var(t), S.T (Wtrans)xW = 1

where t = $\sum$ XW  over all samples

X = input matrix
W = weight vector
t = scores vector (this will provide scores to each of the variables in the input)

max $\sum$ ((t - tmean)(t-tmean))/(n-1)

Now since, we have scaled the data so the t has 0 mean and unit variance, this expression just reduces to

max t.t

using the definition of t we have

max (Wtrans.X)(Xtrans.W)

S.T WtransW = 1

We have something golden here. Let's see what is it and how. We can apply Lagrange's multiplier here to get this into an optimization problem to solve.

L = F - pG
p = Lagrange multipler
G = WtransW

We want to maximize

L = (Wtrans.Xtrans)(X.W) - p(WtransW)

dL/dW = 0 and dL/dX = 0

dL/dW = XtransXW - pW = 0

dL/dX = WtransW = 0


This gives us,

(XtransX)W = pW ....we found something golden!!!

This essentially reduces to the Eigenvalue problem. If you are not aware of Eigen values and Eigen vectors, If I have to simply put for a matrix A that is invertible and eigen vector v with the eigen value p relates,
Av = pv

Since we autoscaled our data, XtransX is simply the covariance matrix of X, which has two good properties 
1. It is symmetric
2. It is positive definite

Solving for eigen value, the largest eigen value gives us the optimal weight for a single principle component. This naturally extends to multiple principle components so as to why stop at the largest eigen value, and we can go on to have k number of principle components in that manner.





